A SQL script starts with a BEGIN … END compound statement. 

Control-flow like IF / WHILE / FOR only works inside that compound statement. 

In [0]:
BEGIN
  -- local variables (must come first)
  DECLARE watermark TIMESTAMP;
  DECLARE inserted  BIGINT;

  -- demo source
  CREATE OR REPLACE TABLE demo_src AS
  SELECT * FROM VALUES
    (1, TIMESTAMP '2025-12-01 00:00:00'),
    (2, TIMESTAMP '2025-12-02 00:00:00') AS t(id, event_ts);

  -- target table (empty on first run)
  CREATE TABLE IF NOT EXISTS demo_tgt (id INT, event_ts TIMESTAMP);

  -- get current watermark from target
  SET watermark = (
    SELECT COALESCE(MAX(event_ts), TIMESTAMP '1900-01-01 00:00:00')
    FROM demo_tgt
  );

  -- load only new rows
  INSERT INTO demo_tgt
  SELECT *
  FROM demo_src
  WHERE event_ts > watermark;

  -- simple “rows inserted” estimate for the demo
  SET inserted = (SELECT COUNT(*) FROM demo_src WHERE event_ts > watermark);

  IF inserted = 0 THEN
    SELECT 'No new rows. watermark=' || CAST(watermark AS STRING) AS status;
  ELSE
    SELECT
      'Loaded ' || CAST(inserted AS STRING) ||
      ' rows. new_watermark=' || CAST((SELECT MAX(event_ts) FROM demo_tgt) AS STRING) AS status;
  END IF;
END;


In [0]:
BEGIN
  DECLARE d     DATE DEFAULT DATE '2025-12-01';
  DECLARE end_d DATE DEFAULT DATE '2025-12-03';

  -- demo input events
  CREATE OR REPLACE TABLE demo_events AS
  SELECT * FROM VALUES
    (DATE '2025-12-01', 'a'),
    (DATE '2025-12-01', 'b'),
    (DATE '2025-12-02', 'c') AS t(dt, payload);

  CREATE OR REPLACE TABLE demo_daily_counts (dt DATE, cnt BIGINT);

  WHILE d <= end_d DO
    INSERT INTO demo_daily_counts
    SELECT d AS dt, COUNT(*) AS cnt
    FROM demo_events
    WHERE dt = d;

    SET d = DATE_ADD(d, 1);
  END WHILE;

  SELECT * FROM demo_daily_counts ORDER BY dt;
END;


In [0]:
BEGIN
  DECLARE tab STRING;
  DECLARE cnt BIGINT;

  -- demo tables
  CREATE OR REPLACE TABLE demo_t1 AS SELECT id FROM range(0, 5)  AS t(id);
  CREATE OR REPLACE TABLE demo_t2 AS SELECT id FROM range(0, 12) AS t(id);

  CREATE OR REPLACE TABLE demo_table_counts (table_name STRING, row_count BIGINT);

  FOR r AS
    SELECT table_name
    FROM VALUES ('demo_t1'), ('demo_t2') AS v(table_name)
  DO
    SET tab = r.table_name;
    SET cnt = (SELECT COUNT(*) FROM IDENTIFIER(tab));

    INSERT INTO demo_table_counts VALUES (tab, cnt);
  END FOR;

  SELECT * FROM demo_table_counts ORDER BY table_name;
END;
